In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import scipy.stats as stats
import matplotlib.pyplot as plt

idx = pd.IndexSlice

Your Tasks
Your project will be divided into the following tasks

I. Exploratory Data Analysis

Before making recommendations of any kind, you will need to explore the data you are working with for the project. Dive in to see what you can find. There are some basic, required questions to be answered about the data you are working with throughout the rest of the notebook. Use this space to explore, before you dive into the details of your recommendation system in the later sections.

II. Rank Based Recommendations

To get started in building recommendations, you will first find the most popular articles simply based on the most interactions. Since there are no ratings for any of the articles, it is easy to assume the articles with the most interactions are the most popular. These are then the articles we might recommend to new users (or anyone depending on what we know about them).

III. User-User Based Collaborative Filtering

In order to build better recommendations for the users of IBM's platform, we could look at users that are similar in terms of the items they have interacted with. These items could then be recommended to the similar users. This would be a step in the right direction towards more personal recommendations for the users. You will implement this next.

IV. Content Based Recommendations (EXTRA - NOT REQUIRED)

Given the amount of content available for each article, there are a number of different ways in which someone might choose to implement a content based recommendations system. Using your NLP skills, you might come up with some extremely creative ways to develop a content based recommendation system. You are encouraged to complete a content based recommendation system, but not required to do so to complete this project.

V. Matrix Factorization

Finally, you will complete a machine learning approach to building recommendations. Using the user-item interactions, you will build out a matrix decomposition. Using your decomposition, you will get an idea of how well you can predict new articles an individual might interact with (spoiler alert - it isn't great). You will finally discuss which methods you might use moving forward, and how you might test how well your recommendations are working for engaging users.


Before you submit your work, check the [RUBRIC](https://learn.udacity.com/nanodegrees/nd025/parts/cd0019/lessons/ls11961/concepts/ls11961-project-rubric) (opens in a new tab)to make sure you meet all of the rubric items.

# Things to remember
- [Course 5, chapter 3.6](https://learn.udacity.com/nanodegrees/nd025/parts/cd0019/lessons/27c53b6f-1da3-42e0-805b-51dfee1c61b8/concepts/20d75ef1-8718-4441-bb6b-70603fb8f1c0)
- [Course 5, chapter 3.3](https://learn.udacity.com/nanodegrees/nd025/parts/cd0019/lessons/27c53b6f-1da3-42e0-805b-51dfee1c61b8/concepts/5f35a1d2-0a02-42c0-949d-490a0abc9e17?lesson_tab=lesson)
- [Coutse 5, chapter 7.11](https://learn.udacity.com/nanodegrees/nd025/parts/cd0019/lessons/ls11960/concepts/3824dbbd-f9b2-42e4-9479-046e3bdee1a5?lesson_tab=lesson)

For a binomial distribution, the standard error given some probability of success p and sample size n is:
$$
SE = \sqrt{\frac{p(1-p)}{n}}
$$

When calculating statistical power, the standard error of the null is then:
$$
SE_{null} = \sqrt{\frac{p(1-p) + p(1-p)}{n}}
$$

For SVD:

$$
M = U \sigma V^T
$$

$$ U_{n x k} $$
$$\Sigma_{k x k} $$
$$V^T_{k x m} $$

where:

1. n is the number of users
2. k is the number of latent features to keep (4 for this case) - there can be as many latent features are there are items (like PCA). So in the movie example, with users along the index and movie IDs along columns, this would be, at most, the number of movies.
3. m is the number of movies

Notes:

1. $U$ is a square matrix with the number of rows and columns equaling the number of users. 
2. $V^T$ is also a square matrix with the number of rows and columns equaling the number of items.
3. $\sigma$ is actually returned as just an array with 4 values.  

In order to set up the matrices in a way that they can be multiplied together, we have a few steps to perform: 

1. Turn sigma into a square matrix with the number of latent features we would like to keep. 
2. Change the columns of u and the rows of v transpose to match this number of dimensions. 
If we would like to exactly re-create the user-movie matrix, we could choose to keep all of the latent features.


In [ ]:
def statistical_power(p_null, p_alt, n, alpha = .05, plot = True):
    """
    Compute the power of detecting the difference in two populations with 
    different proportion parameters, given a desired alpha rate.
    
    Input parameters:
        p_null: base success rate under null hypothesis
        p_alt : desired success rate to be detected, must be larger than
                p_null
        n     : number of observations made in each group
        alpha : Type-I error rate
        plot  : boolean for whether or not a plot of distributions will be
                created
    
    Output value:
        power : Power to detect the desired difference, under the null.
    """

    # The example explored is a one-tailed test, with the alternative value greater than the null. 
    # The power computations performed in the first part will _not_ work if the alternative proportion 
    # is greater than the null, e.g. detecting a proportion parameter of 0.88 against a null of 0.9. 
    # You might want to try to rewrite the code to handle that case! The same issue should not show 
    # up for the second approach, where we directly compute the sample size.    
    
    # assert p_null > p_alt, "Cannot handle situaiton where p_alt is less than p_null"

    # Compute the power
    se_null = np.sqrt((p_null * (1-p_null) + p_null * (1-p_null)) / n)
    null_dist = stats.norm(loc = 0, scale = se_null)
    p_crit = null_dist.ppf(1 - alpha)
    
    se_alt  = np.sqrt((p_null * (1-p_null) + p_alt  * (1-p_alt) ) / n)
    alt_dist = stats.norm(loc = p_alt - p_null, scale = se_alt)
    beta = alt_dist.cdf(p_crit)
    
    if plot:
        # Compute distribution heights
        low_bound = null_dist.ppf(.01)
        high_bound = alt_dist.ppf(.99)
        x = np.linspace(low_bound, high_bound, 201)
        y_null = null_dist.pdf(x)
        y_alt = alt_dist.pdf(x)

        # Plot the distributions
        plt.plot(x, y_null)
        plt.plot(x, y_alt)
        plt.vlines(p_crit, 0, np.amax([null_dist.pdf(p_crit), alt_dist.pdf(p_crit)]),
                   linestyles = '--')
        plt.fill_between(x, y_null, 0, where = (x >= p_crit), alpha = .5)
        plt.fill_between(x, y_alt , 0, where = (x <= p_crit), alpha = .5)
        
        plt.legend(['null','alt'])
        plt.xlabel('difference')
        plt.ylabel('density')
        plt.show()
    
    # return power
    return (1 - beta)

def experiment_size(p_null, p_alt, alpha = .05, beta = .20):
    """
    Compute the minimum number of samples needed to achieve a desired power
    level for a given effect size.
    
    Input parameters:
        p_null: base success rate under null hypothesis
        p_alt : desired success rate to be detected
        alpha : Type-I error rate
        beta  : Type-II error rate
    
    Output value:
        n : Number of samples required for each group to obtain desired power
    """
    
    # Get necessary z-scores and standard deviations (@ 1 obs per group)
    z_null = stats.norm.ppf(1 - alpha)
    z_alt  = stats.norm.ppf(beta)
    sd_null = np.sqrt(p_null * (1-p_null) + p_null * (1-p_null))
    sd_alt  = np.sqrt(p_null * (1-p_null) + p_alt  * (1-p_alt) )
    
    # Compute and return minimum sample size
    p_diff = p_alt - p_null
    n = ((z_null*sd_null - z_alt*sd_alt) / p_diff) ** 2
    return np.ceil(n)

In [ ]:
statistical_power(
    p_null=0.15, p_alt=0.175, n=520, alpha = .05)

In [ ]:
experiment_size(.1, .12)

In [ ]:
# Statistical power - really not sure how they arrice at their specific values, especially the "rate" decimals.



alpha = 0.05
power = 0.8
Z_alphadiv2 = stats.norm.ppf(1-(alpha/2))
Z_beta = stats.norm.ppf(power)

start_days = 520
p0 = 0.15
p1 = 0.175

num_term1 = np.power(Z_alphadiv2 + Z_beta, 2)
num_term2 = p0 * (1-p0) + p1 * (1-p1)
num = num_term1 * num_term2
den = np.power(p1-p0, 2) * start_days

n = num/den

print(n)

# CRISP-DM Flow

## Business Understanding

## Data Understanding

## Data Preparation

## Data Modeling

## Result Evaluation

## Deployment